In [16]:
import imageio 
filename = 'data/testvideo_eating1.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')

i  = 0
for image in vid.iter_data():
    i+=1
    if i%10 == 0: imageio.imwrite('frames/{}.jpg'.format(i),image)
        
# import imageio
# import visvis as vv

# reader = imageio.get_reader('<video0>')
# t = vv.imshow(reader.get_next_data(), clim=(0, 255))
# for im in reader:
#     vv.processEvents()
#     t.SetData(im)

In [35]:
import numpy as np
import torchvision.transforms as transforms
import torch

from src.extractor_utils import (predict, save_prediction)
from src.Dataset import KaggleSafeDriverDataset
from src.imgnet_utils import denormalize
from src.data_loader import _create_dataLoader
from src.data_loader import _create_dataLoader
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

In [39]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)
from src.extractor_utils import (predict, getPrediction,save_prediction,
                                 load_prediction, torch_summarize,RandomSearch)

from utils.utils import (create_submission ,metrics2csv)

# Choosing resources to extrac the features

In [18]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

1 GPU's available:

cpu_count: 8


In [19]:
use_gpu = True
use_DataParalel= False
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))

    else:
        print('Using only one GPU')

if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using only one GPU


In [20]:
path2test = "frames" 

# Image transformation

In [21]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'test': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

/home/yifang/anaconda3/envs/python3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:176: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


# Creating Dataset & DataLoader

In [22]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [23]:
dsets = {
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['test'],use_only=use_only, is_test=True),
}

In [24]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [25]:
dset_sizes = {x: len(dsets[x]) for x in ['test']} 
dset_classes = len(dsets['test'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

(74, {'test': 74})

In [26]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

Using DenseNet model


In [27]:
if use_gpu:
    print('Using GPU')# {}'.format(device_id))
    model.cuda()
    convnet = model.mrnc 

if use_CPU:
    print("Using CPU's")
    convnet = model.mrnc

Using GPU


# Extracting the features

In [32]:
#extract features from images
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=use_gpu)
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

predict: 2/2 ok
Execution time 1.00 s
torch.Size([74]) torch.Size([74, 2208])


# Make Prediction

In [42]:
conv_dset= {
    'test':  TensorDataset(convOutput_test['pred'], convOutput_test['true'])
}
dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)



# num_epochs = 50

# loss_fn = nn.CrossEntropyLoss()
# optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3,weight_decay=0)

# params = {'model' : model.mrnd, 
#     'criterion': loss_fn,  
#     'optimizer': optimizer, 
#     'callbacks': [savebest, ptt.AccuracyMetric()] #ptt.PlotCallback(),
# }
# params['callbacks'].append(ptt.PrintCallback())
# scheduler = StepLR(optimizer, step_size=5, gamma=0.55)
# trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,lr_scheduler=scheduler,**params)
path2saveModel = './models/'+model_name+'RandomSearch'+'DistractDriver' 
model.mrnd.load_state_dict(torch.load(path2saveModel + '.model', map_location=lambda storage, loc: storage))
# make a prediction
result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)
predictions_out = {'test': (result_test['pred'], result_test['true'], model_name)}
# save prediction
path2results = './results/'
save_prediction(path2results,predictions_out)

predict: 2/2 ok
Execution time 0.00 s

Saving DenseNet test
Saved in:./results/test/DenseNet.npz


In [55]:
predictions_out['test'][0][9:,:]



Columns 0 to 7 
 -1.9912   2.5178   2.2675   3.2014   0.4789  -3.4275   7.7902  -6.8293
 -3.7951   1.8742  -5.3601   2.3707  -2.1447   4.2876   1.2077  -1.4793
  3.6310   7.6535  -4.8039  -3.7464  -5.1389   5.6956  -5.4594  -0.0135
 -0.0664   3.6334  -4.4866  -0.6831  -5.4756  -0.3897   0.2998  -0.3111
  1.4952  -2.9310  -2.6234   0.0652   2.2749   2.6195   0.0176  -1.3132
 -6.9206  -8.3026  -2.1276  -2.0370   4.6868   0.2801   4.8130  -1.1631
 -1.5386   1.7804  -3.3565   0.1527  -4.0420   3.2964   0.5270  -1.6409
 -1.9597   0.1089  -4.0956   2.7950  -3.7383   1.8269  -1.1254   3.3506
 -6.6974  -1.2856   4.8005   3.7027   2.1396  -4.3035   8.2384  -6.0202
  1.6423  -1.7525  -2.8793  -4.8347  -0.2570   2.6072  -4.0379  -1.0506
 -1.0892   4.0253  -2.5844  -3.6867  -5.3451   0.3265  -3.0611  -0.4431
  0.3043  -2.4144  -0.5293  -0.9771  -3.1491   5.2794   0.7433  -0.9931
  3.8289  -2.2835  -1.6925  -3.2535   0.5250   5.0542  -4.7588  -1.5687
 -2.3960   5.5308  -1.0720   6.9864  -1.2374  

In [56]:
getPrediction(result_test)

{'pred': array([1, 9, 3, 5, 8, 5, 5, 1, 2, 6, 5, 1, 1, 5, 8, 8, 8, 6, 9, 9, 5, 5,
        3, 8, 3, 5, 5, 6, 9, 1, 3, 9, 1, 3, 5, 7, 8, 2, 2, 8, 8, 4, 5, 5,
        8, 5, 4, 1, 9, 8, 1, 5, 3, 2, 2, 8, 6, 1, 1, 0, 8, 2, 8, 9, 6, 1,
        2, 6, 1, 5, 8, 8, 5, 5]),
 'true': array([150, 490, 390, 320, 100, 180, 610, 570, 690, 580, 360, 130, 220,
        650, 530, 310, 380, 260, 480, 500, 660,  40, 230,  10, 300, 240,
        520, 370, 410, 400, 290, 190,  60, 420, 460,  70,  30, 700, 710,
         80, 740, 270, 340, 140, 470, 350, 620, 540, 550, 430, 110, 560,
        630, 600, 730,  50, 250, 120,  90, 160, 200, 170, 590, 510, 280,
        330, 720, 640, 680, 670, 440,  20, 450, 210])}